### Github上でGoogleが管理しているリポジトリの情報をスクレイピングして下記の情報を取得する
* リポジトリ名
* 主要な言語
* スターの数
### 上記のデータを保存するためのDBを作成し，スクレイピングしたデータを保存する
* GithubのAPIは使わないこと（スクレイピングの課題なので）
* ちゃんと`time.sleep(1)`入れてね
### 保存したデータをSELECT文で表示する
### ソースコードはGithubフローに従って管理すること
1. ブランチ作成（add-exercise-1）
2. ステージング
3. コミット（コミットはキリのいいところでこまめにすること）
4. プッシュ
5. プルリク
6. コードレビュー，マージ（プルリクは自分で処理してよい）

In [46]:
# Webスクレイピングに最低限必要なライブラリをインポート
import requests
from bs4 import BeautifulSoup
import time
import sqlite3

In [47]:
# 初期設定
URL = "https://github.com" # アクセスするgithubのURL
repositories_url = "https://github.com/orgs/google/repositories"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(repositories_url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
time.sleep(1) # クローリングの際の負荷を削減するための待ち時間
empty_list = []
visited = set()

In [48]:
# リポジトリ名
for a in soup.find_all("a", href=True): # aタグをすべて取得
    href = a["href"] # href属性を取得
    if href.startswith("/google/") and href.count("/") == 2: # googleが管理しているリポジトリへのリンクのみを抽出
        if href not in visited: # 重複回避
            visited.add(href) # 訪問済みに入れる
            empty_list.append(a) # リストに追加

print(empty_list) # リポジトリ名リストの表示

# 主要言語
language_list = [] # 主要言語リスト
for repo in empty_list:
    repo_url = URL + repo["href"] # 各リポジトリのURLを作成
    response = requests.get(repositories_url, headers=headers) # 各リポジトリのページにアクセス
    time.sleep(1) # クローリングの際の負荷を削減するための待ち時間

    language_tag = soup.find("span", attrs={"itemprop": "programmingLanguage"}) # 主要言語タグを取得
    if language_tag: 
        language = language_tag.text.strip() # 主要言語を取得する
    else: # 主要言語がない場合の処理
        language = "Not specified" # 主要言語が指定されていない場合の表示
    language_list.append((repo.text.strip(), language)) # リストに追加

print(language_list) # 主要言語リストの表示

# スター数
star_list = [] # スター数リスト
for repo in empty_list:
    repo_url = URL + repo["href"] # 各リポジトリのURLを作成
    response = requests.get(repositories_url, headers=headers) # 各リポジトリのページにアクセス
    time.sleep(1) # クローリングの際の負荷を削減するための待ち時間

    star_tag = soup.find("a", class_="social-count js-social-count") # スター数タグを取得
    if star_tag:
        stars = star_tag.text.strip() # スター数を取得
    else:
        stars = "0" # スター数がない場合の処理
    star_list.append((repo.text.strip(), stars)) # リストに追加

print(star_list) # スター数リストの表示

[<a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/secure-aggregation"><span>secure-aggregation</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/perfetto"><span>perfetto</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/adk-java"><span>adk-java</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/ksp"><span>ksp</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/XNNPACK"><span>XNNPACK</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/koladata"><span>koladata</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/skia-buildbot"><span>skia-buildbot</span></a>, <a class="Title-module__anchor--GmXUE Title-module__inline--oM0P7" href="/google/merchant-api-samples"><span>merchant-api-samples</span></a>, <a class="Tit

In [49]:
path = ''
db_name = 'kadai.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのmカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # テーブルの作成
    sql = 'CREATE TABLE kadai (name TEXT, language TEXT, stars INT);'

    # SQL文の実行
    cur.execute(sql)
    print('テーブルを作成しました。')
    # テーブル名：kadai

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()

エラーが発生しました: table kadai already exists


In [50]:
path = ''
db_name = 'car.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのmカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # データの挿入
    # INSERT INTO テーブル名　VALUES (列に対応したデータをカンマ区切りで);
    # 文字列データを挿入する場合は、文字列をシングルクォーテーションで挟む
    sql = "INSERT INTO kadai (name, language, stars) VALUES (?, ?, ?);"

    kadai = []
    for name, language, stars in zip(empty_list, language_list, star_list):

        # 名前の整形
        name_str = name.text.strip() if hasattr(name, "text") else str(name)

        # ★ タプル対策
        if isinstance(stars, tuple):
            stars = stars[0]
        stars_str = str(stars)
        stars_clean = stars_str.replace("★", "").replace(",", "").strip()
        stars_int = int(stars_clean) if stars_clean.isdigit() else 0

    kadai.append((name_str, language, stars_int))

    # SQL文の実行
    cur.executemany(sql, kadai)

    # 変更をDBに反映させる
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()

エラーが発生しました: no such table: kadai


In [51]:
path = ''
db_name = 'car.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL(RDBを操作するための言語)を実行するためのmカーソルオブジェクトを取得
    cur = conn.cursor()

    # データを参照するSQL
    # SELECT * FROM テーブル名;
    # * の部分は、取得したい列の名前を,区切りで指定することもできる
    sql = "SELECT * FROM kadai;"

    # SQL文の実行
    cur.execute(sql)


except sqlite3.Error as e:
    print('エラーが発生しました:', e)

else:
    for row in cur:
        id, name, language, stars = row
        print(id, name, language, stars)

finally:
    # DBへの接続を閉じる
    conn.close()

エラーが発生しました: no such table: kadai


# テーブルを作成したがデータの挿入のところでテーブルが存在しないというエラーを吐いてしまいそれ以降進められませんでした。それよりも手前で課題の採点をしていただきたいです。よろしくお願いします。